In [1]:
import pandas as pd
import json
import psycopg2 as p2
from datetime import date,datetime,timedelta

In [2]:
rapid_reorder_control=pd.read_csv('data\Rapid_Reorder_Control_export.csv')
rapid_reorder_recipients=pd.read_csv('data\Rapid_Reorder_Recipients_export.csv')

In [3]:
def aws_connection(file_name:str=''):
    # Connecting to Amazon Redshift DB
    # authentication credentials to connect to redshift db from json file
    with open('./cred/{fl_name}.json'.format(fl_name = file_name)) as redshift_connection:
        aws_creds = json.load(redshift_connection)

    # connect to redshift:
    con = p2.connect(
            dbname= aws_creds['dbname'],
            host= aws_creds['host'],
            port= aws_creds['port'],
            user= aws_creds['user'],
            password= aws_creds['password'])
    return con

In [4]:
def query_aws(file_name: str='', query_path: str='./queries/', query_name: str = ''):
    '''Returns DataFrame based on a provided query '''
    conn = aws_connection(file_name=file_name)
    query = open(query_path + query_name).read()
    data = pd.read_sql(sql=query, con=conn)
    return data

### Reformating the control group, joined with the orders for the control group customers

In [5]:
all_orders_from_2019=query_aws(file_name='the_phoenix_connection', query_name='rapid_reorder_eda.sql')

In [6]:
controL_group_orders=pd.merge(all_orders_from_2019,rapid_reorder_control,how='inner',left_on='email',right_on='email')
controL_group_orders['unsubscribed_from_emails_at']=controL_group_orders['unsubscribed_from_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))
controL_group_orders['opted_in_to_emails_at']=controL_group_orders['opted_in_to_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))

In [7]:
test_group_orders=pd.merge(all_orders_from_2019,rapid_reorder_recipients,how='inner',left_on='email',right_on='email')
test_group_orders['unsubscribed_from_emails_at']=test_group_orders['unsubscribed_from_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))
test_group_orders['opted_in_to_emails_at']=test_group_orders['opted_in_to_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))

### Bringing promo codes into the picture

In [8]:
promo_codes=query_aws(file_name='the_phoenix_connection', query_name='rapid_reorder_promocodes.sql')

In [15]:
control_group_orders_with_promo_codes=pd.merge(controL_group_orders,promo_codes,how='left',on='promo_code_id')
test_group_orders_with_promo_codes=pd.merge(test_group_orders,promo_codes,how='left',on='promo_code_id')

#### Filtering only the promo codes we want to analyze: 
###### 'PIZZAGAIN','ROUNDTWO','ONEMORESLICE','STILLHUNGRY','ONEMOREPIE','ONCEAGAIN','NEVERENOUGH','REPEATZZA2','REPEATZZA5','DOUBLEDOWN'

In [59]:
control_group_focus_promo_codes = control_group_orders_with_promo_codes[control_group_orders_with_promo_codes['promo_code_tag'].isin(['PIZZAGAIN','ROUNDTWO',
                                                                                                                                      'ONEMORESLICE','STILLHUNGRY',
                                                                                                                                      'ONEMOREPIE','ONCEAGAIN',
                                                                                                                                      'NEVERENOUGH','REPEATZZA2',
                                                                                                                                      'REPEATZZA5','DOUBLEDOWN'])]
test_group_focus_promo_codes = test_group_orders_with_promo_codes[test_group_orders_with_promo_codes['promo_code_tag'].isin(['PIZZAGAIN','ROUNDTWO','ONEMORESLICE',
                                                                                                                             'STILLHUNGRY','ONEMOREPIE','ONCEAGAIN',
                                                                                                                             'NEVERENOUGH','REPEATZZA2','REPEATZZA5',
                                                                                                                             'DOUBLEDOWN'])]

In [60]:
test_group_focus_promo_codes=test_group_focus_promo_codes[['orders_id', 'customer', 'source', 'shop_id', 'date_purchased_pst',
       'mypizza_fees', 'cc_fees', 'email', 'cached_total', 'promo_code_id', 'user_id', 'unsubscribed_from_emails_at',
       'opted_in_to_emails_at', 'promo_code_tag', 'description', 'department', 'promo_flat_amount']]

In [143]:
test_group_focus_promo_codes['days_between_opt_in_and_order']=test_group_focus_promo_codes['date_purchased_pst'] - test_group_focus_promo_codes['opted_in_to_emails_at']
control_group_focus_promo_codes['days_between_opt_in_and_order']=control_group_focus_promo_codes['date_purchased_pst'] - control_group_focus_promo_codes['opted_in_to_emails_at']

C:\Users\asaid\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Summmary for the data that we have

In [120]:
input_summary=pd.DataFrame(columns=['control_vs_test','Unique_users','Users_that_opted_in_to_emails','percent_of_user_that_opted_in'])
inputs=pd.DataFrame(columns=['control_vs_test','Unique_users','Users_that_opted_in_to_emails'])
inputs=pd.DataFrame([
    ['control', rapid_reorder_control['email'].nunique(), rapid_reorder_control['opted_in_to_emails_at'].notnull().sum(),
                      rapid_reorder_control['opted_in_to_emails_at'].notnull().sum()/rapid_reorder_control['email'].nunique()],
                     ['test', rapid_reorder_recipients['email'].nunique(), rapid_reorder_recipients['opted_in_to_emails_at'].notnull().sum(),
                      rapid_reorder_recipients['opted_in_to_emails_at'].notnull().sum()/rapid_reorder_recipients['email'].nunique()]
                    ]
    ,columns=['control_vs_test','Unique_users','Users_that_opted_in_to_emails','percent_of_user_that_opted_in'])
input_summary = input_summary.append(inputs,ignore_index=True)
input_summary

,control_vs_test,Unique_users,Users_that_opted_in_to_emails,percent_of_user_that_opted_in
0,control,339443,25398,0.074823
1,test,333551,25442,0.076276


### within 4 days after opt-in

In [125]:
four_days = timedelta(days=4)
zero_days = timedelta(days=0)

In [144]:
test_group_focus_promo_codes_within_4_days_after_opt_in = test_group_focus_promo_codes[(test_group_focus_promo_codes['days_between_opt_in_and_order'] < four_days)
                             &
                            (test_group_focus_promo_codes['days_between_opt_in_and_order'] > zero_days)]

control_group_focus_promo_codes_within_4_days_after_opt_in = control_group_focus_promo_codes[(control_group_focus_promo_codes['days_between_opt_in_and_order'] < four_days)
                             &
                            (control_group_focus_promo_codes['days_between_opt_in_and_order'] > zero_days)]

In [146]:
test_group_focus_promo_codes_within_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique()

promo_code_tag
DOUBLEDOWN      10
NEVERENOUGH     13
ONCEAGAIN        3
ONEMOREPIE      14
ONEMORESLICE    15
PIZZAGAIN       15
REPEATZZA2      10
REPEATZZA5      26
ROUNDTWO        18
STILLHUNGRY     14
Name: customer, dtype: int64

In [147]:
control_group_focus_promo_codes_within_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique()

promo_code_tag
REPEATZZA5    1
Name: customer, dtype: int64

### after 4 days since opt-in

In [148]:
test_group_focus_promo_codes_after_4_days_after_opt_in = test_group_focus_promo_codes[(test_group_focus_promo_codes['days_between_opt_in_and_order'] > four_days)]

control_group_focus_promo_codes_after_4_days_after_opt_in = control_group_focus_promo_codes[(control_group_focus_promo_codes['days_between_opt_in_and_order'] > four_days)]

In [149]:
test_group_focus_promo_codes_after_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique()

promo_code_tag
DOUBLEDOWN      56
NEVERENOUGH     50
ONCEAGAIN       50
ONEMOREPIE      57
ONEMORESLICE    63
PIZZAGAIN       64
REPEATZZA2      48
REPEATZZA5      50
ROUNDTWO        48
STILLHUNGRY     41
Name: customer, dtype: int64

In [150]:
control_group_focus_promo_codes_after_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique()

promo_code_tag
DOUBLEDOWN    1
REPEATZZA2    1
REPEATZZA5    1
Name: customer, dtype: int64

### before opt-in

In [152]:
test_group_focus_promo_codes_before_opt_in = test_group_focus_promo_codes[(test_group_focus_promo_codes['days_between_opt_in_and_order'] < zero_days)]

control_group_focus_promo_codes_before_opt_in = control_group_focus_promo_codes[(control_group_focus_promo_codes['days_between_opt_in_and_order'] < zero_days)]

In [153]:
test_group_focus_promo_codes_before_opt_in.groupby('promo_code_tag').customer.nunique()

promo_code_tag
DOUBLEDOWN      106
NEVERENOUGH     148
ONCEAGAIN       102
ONEMOREPIE      121
ONEMORESLICE    130
PIZZAGAIN       146
REPEATZZA2      120
REPEATZZA5      166
ROUNDTWO        112
STILLHUNGRY     111
Name: customer, dtype: int64

In [154]:
control_group_focus_promo_codes_before_opt_in.groupby('promo_code_tag').customer.nunique()

promo_code_tag
NEVERENOUGH     5
ONCEAGAIN       1
ONEMORESLICE    2
REPEATZZA2      2
REPEATZZA5      4
ROUNDTWO        2
STILLHUNGRY     2
Name: customer, dtype: int64